In [ ]:
!pip install pyldavis pandas==1.5.1 ftfy cleantext wordcloud bertopic umap

In [2]:
import warnings

warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import os
import re
from wordcloud import WordCloud
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim.corpora as corpora

from PIL import Image
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim_models
from ftfy import fix_encoding
from cleantext import clean
#from bertopic import BERTopic
#from umap import UMAP
from sklearn.feature_extraction.text import CountVectorizer
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt
import warnings


warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
stop_words = stopwords.words('english')
addn_stopwords = ['app','really','snapchat', 'google', 'assistant','please','dark','mode','update','wrong','something',
                  'went','say','ever','android','ios','replika','chatgpt','gpt','snap','filter','also','ai', 'ass']
stop_words.extend(addn_stopwords)

In [4]:
path = "/kaggle/input/ai-companions-app-reviews/reviews.csv"
data = pd.read_csv(path)
data.drop('Unnamed: 0', axis=1, inplace=True)
data['date']  = pd.to_datetime(data['date'])

In [5]:
filtered_data = data[data['rev_length']>=10]
filtered_data['review'] = filtered_data['review'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
filtered_data['review'] = filtered_data['review'].apply(lambda x: x.lower())

In [6]:
positive_revs = filtered_data[filtered_data['feedback']=='positive']
negative_revs = filtered_data[filtered_data['feedback']=='negative']

after_21_revs = filtered_data[filtered_data['date']>'2021']
before_21_revs = filtered_data[filtered_data['date'] <= '2021']

In [7]:
all_words = ' '.join(filtered_data['review'].values)

before_21 = ' '.join(before_21_revs['review'].values)
after_21 = ' '.join(after_21_revs['review'].values)

negative = ' '.join(negative_revs['review'].values)
positive = ' '.join(positive_revs['review'].values)

In [8]:
#LDA
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]

In [9]:
all_words = filtered_data.review.values.tolist()
pos_words = positive_revs.review.values.tolist()
neg_words = negative_revs.review.values.tolist()
after_21_words = after_21_revs.review.values.tolist()
before_21_words = before_21_revs.review.values.tolist()


data_words = remove_stopwords(list(sent_to_words(all_words)))
pos_data_words = remove_stopwords(list(sent_to_words(pos_words)))
neg_data_words = remove_stopwords(list(sent_to_words(neg_words)))
after_21_data_words = remove_stopwords(list(sent_to_words(after_21_words)))
before_21_data_words = remove_stopwords(list(sent_to_words(before_21_words)))

In [10]:
id2_word_all = corpora.Dictionary(data_words)
texts_all = data_words
corpus_all = [id2_word_all.doc2bow(text) for text in texts_all]

id2_word_pos = corpora.Dictionary(pos_data_words)
texts_pos = pos_data_words
corpus_pos = [id2_word_pos.doc2bow(text) for text in texts_pos]

id2_word_neg = corpora.Dictionary(neg_data_words)
texts_neg = neg_data_words
corpus_neg = [id2_word_neg.doc2bow(text) for text in texts_neg]

id2_word_after_21 = corpora.Dictionary(after_21_data_words)
corpus_after_21 = [id2_word_after_21.doc2bow(text) for text in after_21_data_words]

id2_word_before_21 = corpora.Dictionary(before_21_data_words)
corpus_before_21 = [id2_word_before_21.doc2bow(text) for text in before_21_data_words]

In [11]:
def fit_lda(corpus, id2word, num_topics=10):
  lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                        id2word=id2word,
                                        num_topics=num_topics)
  pprint(lda_model.print_topics())
  doc_lda = lda_model[corpus]
  return (lda_model, corpus, id2word)

In [12]:
all_lda = fit_lda(corpus_all, id2_word_all)

[(0,
  '0.031*"like" + 0.016*"voice" + 0.010*"use" + 0.008*"time" + 0.008*"im" + '
  '0.008*"dont" + 0.007*"good" + 0.007*"command" + 0.007*"using" + '
  '0.007*"ask"'),
 (1,
  '0.023*"dont" + 0.016*"know" + 0.015*"hey" + 0.015*"get" + 0.012*"screen" + '
  '0.009*"even" + 0.009*"help" + 0.009*"time" + 0.008*"notifications" + '
  '0.008*"try"'),
 (2,
  '0.016*"voice" + 0.016*"filters" + 0.013*"like" + 0.012*"good" + '
  '0.010*"love" + 0.010*"use" + 0.009*"music" + 0.009*"many" + 0.008*"cant" + '
  '0.007*"still"'),
 (3,
  '0.014*"like" + 0.012*"good" + 0.010*"answer" + 0.010*"great" + 0.009*"time" '
  '+ 0.009*"questions" + 0.009*"would" + 0.009*"helpful" + 0.008*"answers" + '
  '0.008*"ask"'),
 (4,
  '0.029*"phone" + 0.020*"working" + 0.018*"work" + 0.015*"doesnt" + '
  '0.013*"fix" + 0.013*"use" + 0.012*"even" + 0.011*"open" + 0.011*"cant" + '
  '0.009*"works"'),
 (5,
  '0.014*"im" + 0.014*"send" + 0.013*"ive" + 0.011*"problem" + 0.011*"using" + '
  '0.011*"like" + 0.010*"make" + 0.0

In [18]:
pyLDAvis.enable_notebook()
all_viz = pyLDAvis.gensim_models.prepare(all_lda[0], all_lda[1], all_lda[2])



NameError: name 'all_vis' is not defined

In [19]:
pyLDAvis.display(all_viz)

In [14]:
pos_lda = fit_lda(corpus_pos, id2_word_pos)
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(pos_lda[0], pos_lda[1], pos_lda[2])

[(0,
  '0.013*"great" + 0.010*"im" + 0.010*"need" + 0.009*"like" + 0.009*"use" + '
  '0.009*"help" + 0.009*"work" + 0.007*"get" + 0.007*"helpful" + 0.007*"dont"'),
 (1,
  '0.025*"good" + 0.021*"use" + 0.017*"filters" + 0.013*"great" + 0.012*"love" '
  '+ 0.011*"best" + 0.009*"would" + 0.008*"like" + 0.008*"better" + '
  '0.008*"voice"'),
 (2,
  '0.027*"good" + 0.022*"filters" + 0.022*"like" + 0.021*"love" + '
  '0.011*"great" + 0.011*"best" + 0.010*"thank" + 0.009*"nice" + '
  '0.009*"application" + 0.009*"make"'),
 (3,
  '0.019*"like" + 0.017*"great" + 0.016*"add" + 0.015*"would" + 0.015*"good" + '
  '0.014*"filters" + 0.010*"games" + 0.010*"option" + 0.010*"one" + '
  '0.010*"video"'),
 (4,
  '0.018*"snaps" + 0.016*"like" + 0.016*"good" + 0.015*"chat" + 0.014*"voice" '
  '+ 0.011*"cant" + 0.009*"great" + 0.009*"best" + 0.008*"would" + '
  '0.007*"work"'),
 (5,
  '0.047*"like" + 0.021*"love" + 0.018*"friends" + 0.018*"good" + 0.016*"talk" '
  '+ 0.011*"great" + 0.011*"best" + 0.009*"d

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.008342 -0.000627       1        1  17.475528
6      0.101814 -0.001375       2        1  11.833770
0     -0.068863 -0.050384       3        1  11.535504
1     -0.064120  0.001808       4        1  10.831738
8      0.025499 -0.041581       5        1   9.783956
7      0.020849  0.024664       6        1   9.582598
2      0.003772  0.015273       7        1   9.335987
4      0.043778 -0.035404       8        1   6.839781
3     -0.019350  0.120464       9        1   6.682157
9     -0.035037 -0.032835      10        1   6.098981, topic_info=         Term          Freq         Total Category  logprob  loglift
11    filters   6758.000000   6758.000000  Default  30.0000  30.0000
6500    snaps   1605.000000   1605.000000  Default  29.0000  29.0000
15       love  19531.000000  19531.000000  Default  28.0000  28.0000
117      like  25390.000000  25390.000000  Default  27.0000  27.0000
0         add   4596.000000   4596.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
102    things    395.882082   4429.427111  Topic10  -5.3027   0.3821
217      know    406.491122   4946.597525  Topic10  -5.2763   0.2982
117      like    588.243873  25390.292281  Topic10  -4.9067  -0.9679
454      cant    428.255143   6422.868030  Topic10  -5.2241   0.0891
147       fun    346.923700   5293.278554  Topic10  -5.4348   0.0720

[880 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
3410      2  0.982852     aa
7720      3  0.958282    aap
7720      4  0.017344    aap
7720      8  0.023126    aap
8922      6  0.938735    abd
...     ...       ...    ...
138       8  0.064629  years
138       9  0.008463  years
138      10  0.066168  years
8811      2  0.122092    yrs
8811      5  0.869909    yrs

[2870 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 7, 1, 2, 9, 8, 3, 5, 4, 10])

In [15]:
neg_lda = fit_lda(corpus_neg, id2_word_neg)
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(neg_lda[0], neg_lda[1], neg_lda[2])

[(0,
  '0.017*"dont" + 0.015*"want" + 0.013*"like" + 0.012*"good" + 0.012*"bitmoji" '
  '+ 0.009*"every" + 0.009*"get" + 0.008*"fix" + 0.008*"im" + '
  '0.008*"annoying"'),
 (1,
  '0.013*"answer" + 0.012*"ask" + 0.012*"like" + 0.010*"language" + '
  '0.010*"get" + 0.009*"im" + 0.009*"questions" + 0.008*"said" + 0.008*"ok" + '
  '0.008*"doesnt"'),
 (2,
  '0.017*"like" + 0.013*"dont" + 0.012*"cant" + 0.011*"use" + 0.010*"back" + '
  '0.010*"new" + 0.010*"camera" + 0.008*"change" + 0.008*"voice" + '
  '0.008*"feature"'),
 (3,
  '0.013*"phone" + 0.011*"time" + 0.009*"data" + 0.007*"tried" + 0.007*"cache" '
  '+ 0.007*"ask" + 0.007*"assistance" + 0.007*"command" + 0.007*"uninstalled" '
  '+ 0.006*"still"'),
 (4,
  '0.014*"voice" + 0.013*"use" + 0.013*"send" + 0.013*"fix" + 0.011*"im" + '
  '0.011*"doesnt" + 0.011*"working" + 0.011*"problem" + 0.010*"phone" + '
  '0.010*"good"'),
 (5,
  '0.011*"dont" + 0.009*"like" + 0.008*"without" + 0.008*"make" + '
  '0.008*"would" + 0.008*"im" + 0.008*"w

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.045162 -0.038164       1        1  14.730064
4     -0.040167 -0.038118       2        1  13.475212
5      0.079011  0.024335       3        1  12.954004
9     -0.077022 -0.040167       4        1  12.896110
6      0.017731 -0.072901       5        1  11.950210
0      0.027114 -0.030146       6        1   8.639254
8      0.001561 -0.035054       7        1   8.083774
1      0.099343  0.092145       8        1   6.733134
7     -0.145492  0.078217       9        1   5.535925
3     -0.007239  0.059853      10        1   5.002312, topic_info=          Term          Freq         Total Category  logprob  loglift
261      phone  15808.000000  15808.000000  Default  30.0000  30.0000
442    working   9311.000000   9311.000000  Default  29.0000  29.0000
163    account   4155.000000   4155.000000  Default  28.0000  28.0000
3       device   4250.000000   4250.000000  Default  27.0000  27.0000
13062  bitmoji   1768.000000   1768.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
49        like    415.105799  19128.430442  Topic10  -5.3653  -0.8351
87       great    334.105042   6305.268963  Topic10  -5.5824   0.0576
95         see    329.392911   5935.913734  Topic10  -5.5966   0.1037
30       would    334.953045   8572.449675  Topic10  -5.5798  -0.2470
405      every    328.664333   5926.083647  Topic10  -5.5988   0.1032

[841 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
12202      4  0.982282       aa
10964      2  0.034070      aap
10964      9  0.953950      aap
10964     10  0.008517      aap
9893       2  0.007657      acc
...      ...       ...      ...
21531      5  0.113999   zoomed
21531      6  0.004560   zoomed
21531      7  0.004560   zoomed
21531      9  0.004560   zoomed
8787       3  0.993030  zooming

[2973 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 6, 10, 7, 1, 9, 2, 8, 4])

In [16]:
after_21_lda = fit_lda(corpus_after_21, id2_word_after_21)
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(after_21_lda[0], after_21_lda[1], after_21_lda[2])

[(0,
  '0.019*"doesnt" + 0.016*"cant" + 0.014*"work" + 0.012*"dont" + 0.011*"text" '
  '+ 0.011*"use" + 0.010*"voice" + 0.009*"like" + 0.008*"works" + 0.008*"get"'),
 (1,
  '0.028*"like" + 0.015*"would" + 0.014*"good" + 0.014*"want" + 0.012*"dont" + '
  '0.012*"talk" + 0.011*"nice" + 0.008*"much" + 0.008*"get" + 0.008*"friend"'),
 (2,
  '0.030*"best" + 0.021*"good" + 0.018*"like" + 0.011*"great" + 0.008*"love" + '
  '0.007*"information" + 0.007*"language" + 0.006*"helpful" + 0.006*"one" + '
  '0.006*"useful"'),
 (3,
  '0.024*"like" + 0.022*"love" + 0.012*"good" + 0.011*"would" + 0.010*"use" + '
  '0.010*"new" + 0.009*"great" + 0.009*"play" + 0.009*"search" + 0.009*"make"'),
 (4,
  '0.030*"phone" + 0.015*"working" + 0.015*"cant" + 0.013*"even" + '
  '0.013*"dont" + 0.011*"work" + 0.010*"like" + 0.010*"use" + 0.009*"open" + '
  '0.008*"get"'),
 (5,
  '0.018*"im" + 0.018*"problem" + 0.014*"fix" + 0.010*"hey" + 0.009*"cant" + '
  '0.009*"help" + 0.009*"issue" + 0.009*"get" + 0.008*"voice" 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.108463  0.003675       1        1  16.566882
8     -0.081632 -0.055603       2        1  10.386069
7      0.078912  0.017982       3        1  10.143960
0     -0.091276  0.064383       4        1   9.904716
5     -0.112114 -0.032693       5        1   9.841783
1      0.080475 -0.001649       6        1   9.758889
3      0.025751  0.089697       7        1   9.600495
2      0.110773 -0.034282       8        1   9.593969
9      0.033074 -0.050746       9        1   7.535274
6      0.064501 -0.000765      10        1   6.667963, topic_info=        Term          Freq         Total Category  logprob  loglift
261    phone  12548.000000  12548.000000  Default  30.0000  30.0000
526     best   8735.000000   8735.000000  Default  29.0000  29.0000
442  working   8541.000000   8541.000000  Default  28.0000  28.0000
17   problem   7014.000000   7014.000000  Default  27.0000  27.0000
163  account   3930.000000   3930.000000  Default  26.0000  26.0000
..       ...           ...           ...      ...      ...      ...
26      time    670.199783  12594.471490  Topic10  -5.2473  -0.2256
99      work    647.748183  10931.603519  Topic10  -5.2814  -0.1181
476      use    685.550278  17593.607481  Topic10  -5.2247  -0.5372
927   people    535.558275   7055.666050  Topic10  -5.4716   0.1296
66      want    534.775017   9496.199700  Topic10  -5.4731  -0.1689

[858 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
10369      1  0.990171    aa
9412       2  0.179546   aap
9412       3  0.001561   aap
9412       4  0.818103   aap
240        1  0.170158  able
...      ...       ...   ...
5121       3  0.015976  zoom
5121       6  0.007988  zoom
5121       7  0.950588  zoom
5121       9  0.007988  zoom
5121      10  0.007988  zoom

[2979 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 9, 8, 1, 6, 2, 4, 3, 10, 7])

In [17]:
before_21_lda = fit_lda(corpus_before_21, id2_word_before_21)
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(before_21_lda[0], before_21_lda[1], before_21_lda[2])

[(0,
  '0.019*"phone" + 0.019*"cant" + 0.013*"fix" + 0.010*"new" + 0.010*"even" + '
  '0.008*"problem" + 0.008*"working" + 0.008*"camera" + 0.008*"ive" + '
  '0.008*"im"'),
 (1,
  '0.030*"like" + 0.016*"love" + 0.015*"good" + 0.014*"best" + 0.011*"would" + '
  '0.011*"great" + 0.010*"get" + 0.009*"time" + 0.009*"one" + 0.008*"friend"'),
 (2,
  '0.026*"filters" + 0.023*"good" + 0.019*"love" + 0.017*"like" + 0.012*"get" '
  '+ 0.011*"great" + 0.009*"friends" + 0.009*"dont" + 0.009*"talk" + '
  '0.008*"use"'),
 (3,
  '0.014*"dont" + 0.012*"like" + 0.010*"one" + 0.010*"good" + 0.010*"time" + '
  '0.009*"friends" + 0.008*"even" + 0.008*"change" + 0.008*"new" + '
  '0.008*"great"'),
 (4,
  '0.015*"send" + 0.015*"doesnt" + 0.014*"fix" + 0.014*"good" + 0.010*"like" + '
  '0.010*"video" + 0.010*"great" + 0.010*"time" + 0.009*"work" + 0.008*"cant"'),
 (5,
  '0.030*"use" + 0.017*"dont" + 0.013*"im" + 0.013*"good" + 0.011*"like" + '
  '0.008*"phone" + 0.008*"work" + 0.007*"love" + 0.007*"great" + 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.109120  0.016310       1        1  19.956008
1      0.061197  0.003309       2        1  18.159191
4     -0.079501  0.017879       3        1  10.905330
8      0.055740  0.006251       4        1  10.234105
2      0.011509  0.011505       5        1   9.679163
7     -0.027279  0.016663       6        1   8.392912
9      0.063768  0.004910       7        1   7.020613
3      0.038325  0.021666       8        1   6.595377
5     -0.000445  0.031339       9        1   5.350381
6     -0.014194 -0.129832      10        1   3.706921, topic_info=          Term          Freq         Total Category  logprob  loglift
899    filters   5439.000000   5439.000000  Default  30.0000  30.0000
12631    snaps   2295.000000   2295.000000  Default  29.0000  29.0000
210        use   8227.000000   8227.000000  Default  28.0000  28.0000
498       love  12439.000000  12439.000000  Default  27.0000  27.0000
23        like  19322.000000  19322.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
255        ive    186.483100   5024.200694  Topic10  -5.4477   0.0013
64        time    195.012733   7998.187055  Topic10  -5.4030  -0.4189
276        fun    182.386844   4181.856552  Topic10  -5.4699   0.1626
73        dont    185.871195   9181.540562  Topic10  -5.4510  -0.6049
79       great    185.225208   9583.005574  Topic10  -5.4545  -0.6512

[854 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
7989       5  0.955055         aa
10552      2  0.010540        aap
10552      9  0.985529        aap
9885       5  0.921194  absolutly
8417       2  0.955703  accepting
...      ...       ...        ...
1801       7  0.063000         yo
1801       8  0.105001         yo
1801       9  0.598504         yo
5860      10  0.799635   youtuber
10948      7  0.929311        ypu

[2673 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 5, 9, 3, 8, 10, 4, 6, 7])